### Exploring Car Collision Patterns in California (2025 YTD)

#### General Info

Project Type: Exploratory Data Analysis (EDA)

Tools: Python, Pandas, NumPy, Plotly

Date: August 2025

Data Source: California Crash Reporting System (CCRS)

Link: https://data.ca.gov/dataset/ccrs/resource/9f4fc839-122d-4595-a146-43bc4ed16f46 


#### Description
This dataset is from the California Crash Reporting System (CCRS), published by the California Highway Patrol. It contains collision records for 2025 YTD, including dates, locations, involved parties, and injury counts. This data supports traffic safety and risk analysis.

#### Data Cleaning & Preparation

Prior to EDA, data prep included:

•	Standardizing column names for readability

•	Inspecting data types, size, and completeness

•	Handling missing values by dropping columns with >70% nulls

•	Removing irrelevant fields (e.g., sketch notes, milepost details)

•	Verifying Collision_id as a unique identifier

•	Checking redundancy between related fields (e.g., IsHighwayRelated vs IsFreeway)

In [30]:
# Import libraries
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Configure display
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format
%matplotlib inline

# Read dataset
crashes = pd.read_csv('crashes_dataset.csv')

# Preview data
crashes.head()

/var/folders/59/kn5xzsps5mv5g30gtzx52jh40000gn/T/ipykernel_92695/2181682616.py:16: DtypeWarning:

Columns (5,13,14,24,44,54,70) have mixed types. Specify dtype option on import or set low_memory=False.



,_id,Collision Id,Report Number,Report Version,Is Preliminary,NCIC Code,Crash Date Time,Crash Time Description,Beat,City Id,City Code,City Name,County Code,City Is Active,City Is Incorporated,Collision Type Code,Collision Type Description,Collision Type Other Desc,Day Of Week,DispatchNotified,HasPhotographs,HitRun,IsAttachmentsMailed,IsDeleted,IsHighwayRelated,IsTowAway,JudicialDistrict,MotorVehicleInvolvedWithCode,MotorVehicleInvolvedWithDesc,MotorVehicleInvolvedWithOtherDesc,NumberInjured,NumberKilled,Weather 1,Weather 2,Road Condition 1,Road Condition 2,Special Condition,LightingCode,LightingDescription,Latitude,Longitude,MilepostDirection,MilepostDistance,MilepostMarker,MilepostUnitOfMeasure,PedestrianActionCode,PedestrianActionDesc,PreparedDate,Primary Collision Factor Code,Primary Collision Factor Violation,PrimaryCollisionFactorIsCited,PrimaryCollisionPartyNumber,PrimaryRoad,ReportingDistrict,ReportingDistrictCode,ReviewedDate,RoadwaySurfaceCode,SecondaryDirection,SecondaryDistance,SecondaryRoad,SecondaryUnitOfMeasure,SketchDesc,TrafficControlDeviceCode,CreatedDate,ModifiedDate,IsCountyRoad,IsFreeway,CHP555Version,IsAdditonalObjectStruck,NotificationDate,NotificationTimeDescription,HasDigitalMediaFiles,EvidenceNumber,IsLocationReferToNarrative,IsAOIOneSameAsLocation
0,1,4550264,250110005,1,False,4116,2025-01-10T08:28:00,828,A,"1,310.00","4,116.00",San Mateo,41.00,True,True,C,REAR END,NaN,Friday,NotApplicable,True,NaN,NaN,False,False,False,NORTHERN TRAFFIC,C,OTHER MOTOR VEHICLE,NaN,1.00,0.00,CLEAR,NaN,NO UNUSUAL CONDITIONS,NaN,,A,DAYLIGHT,37.57,-122.33,NaN,NaN,NaN,NaN,A,NO PEDESTRIANS INVOLVED,NaN,A,22350 VC,False,1.00,N EL CAMINO REAL,13A,NaN,NaN,A,NaN,NaN,TILTON AVENUE,NaN,NaN,A,2025-01-27T09:04:06,2025-01-27T09:04:06,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,4550263,9580-2025-00214,1,False,9580,2025-01-23T08:45:00,845,171,535.00,"1,942.00",Los Angeles,19.00,True,True,B,SIDE SWIPE,NaN,Thursday,NotApplicable,False,M,NaN,False,True,False,LOS ANGELES SUPERIOR COURT VAN NUYS COURTHOUSE...,C,OTHER MOTOR VEHICLE,NaN,0.00,0.00,CLEAR,NaN,NO UNUSUAL CONDITIONS,NaN,,A,DAYLIGHT,34.20,-118.40,NaN,NaN,NaN,NaN,A,NO PEDESTRIANS INVOLVED,2025-01-23T08:45:00,A,VC 22107,False,1.00,SR-170 S/B FROM SHERMAN WAY,NaN,NaN,2025-01-27T09:03:34,A,S,500.00,SHERMAN WAY,F,NaN,D,2025-01-27T09:03:36,2025-01-27T09:03:36,False,True,4,NaN,2025-01-23T08:47:00,0847,False,NaN,NaN,True
2,3,4550262,9765-2025-00066,1,False,9765,2025-01-15T12:30:00,1230,041,"1,701.00","5,606.00",Santa Paula,56.00,True,True,E,HIT OBJECT,NaN,Wednesday,NotApplicable,False,NaN,NaN,False,True,False,VENTURA SUPERIOR COURT VENTURA DIVISION,J,OTHER OBJECT,METAL OBJECT,0.00,0.00,WIND,NaN,NO UNUSUAL CONDITIONS,NaN,,A,DAYLIGHT,34.33,-119.09,NaN,NaN,NaN,NaN,A,NO PEDESTRIANS INVOLVED,2025-01-15T12:30:00,A,VC 22350,False,1.00,SR-126 E/B,NaN,NaN,2025-01-27T09:03:01,A,W,160.00,TODD LANE,F,NaN,D,2025-01-27T09:03:03,2025-01-27T09:03:03,False,True,4,NaN,2025-01-15T12:35:00,1235,True,EMBEDDED,NaN,True
3,4,4550261,9426-2025-00035,1,False,9426,2025-01-20T12:30:00,1230,460,397.00,"1,500.00",Unincorporated,15.00,True,False,D,BROADSIDE,NaN,Monday,No,False,NaN,NaN,False,False,True,KERN COUNTY SUPERIOR COURT - SHAFTER (MISDEMEA...,C,OTHER MOTOR VEHICLE,NaN,3.00,0.00,CLEAR,NaN,NO UNUSUAL CONDITIONS,NaN,,A,DAYLIGHT,35.62,-119.69,NaN,NaN,NaN,NaN,A,NO PEDESTRIANS INVOLVED,2025-01-20T12:30:00,A,VC 22450(a),False,1.00,SR-46 W/B,NaN,NaN,2025-01-27T09:02:37,A,NaN,NaN,MARTIN AVE,NaN,NaN,A,2025-01-27T09:02:47,2025-01-27T09:02:47,True,False,4,NaN,2025-01-20T12:32:00,1232,False,NaN,NaN,True
4,5,4550260,9580-2025-00216,1,False,9580,2025-01-23T10:30:00,1030,170,535.00,"1,942.00",Los Angeles,19.00,True,True,C,REAR END,NaN,Thursday,NotApplicable,False,NaN,NaN,False,True,False,VAN NUYS COURTHOUSE WEST,C,OTHER MOTOR VEHICLE,NaN,0.00,0.00,CLEAR,NaN,NO UNUSUAL CONDITIONS,NaN,,A,DAYLIGHT,34.20,-118.40,NaN,NaN,NaN,NaN,A,NO PEDESTRIANS INVOLVED,2025-01-23T10:30:00,A,VC 22350,False,1.00,SR-170 N/B TO SHERMAN WAY E/B,NaN,NaN,20

In [32]:
# Check dataset size 
print(f"# rows: {len(crashes):,}")
print(f"# columns: {crashes.shape[1]}")

# rows: 195,655
# columns: 75


In [35]:
# View column names
crashes.columns

Index(['_id', 'Collision Id', 'Report Number', 'Report Version',
       'Is Preliminary', 'NCIC Code', 'Crash Date Time',
       'Crash Time Description', 'Beat', 'City Id', 'City Code', 'City Name',
       'County Code', 'City Is Active', 'City Is Incorporated',
       'Collision Type Code', 'Collision Type Description',
       'Collision Type Other Desc', 'Day Of Week', 'DispatchNotified',
       'HasPhotographs', 'HitRun', 'IsAttachmentsMailed', 'IsDeleted',
       'IsHighwayRelated', 'IsTowAway', 'JudicialDistrict',
       'MotorVehicleInvolvedWithCode', 'MotorVehicleInvolvedWithDesc',
       'MotorVehicleInvolvedWithOtherDesc', 'NumberInjured', 'NumberKilled',
       'Weather 1', 'Weather 2', 'Road Condition 1', 'Road Condition 2',
       'Special Condition', 'LightingCode', 'LightingDescription', 'Latitude',
       'Longitude', 'MilepostDirection', 'MilepostDistance', 'MilepostMarker',
       'MilepostUnitOfMeasure', 'PedestrianActionCode', 'PedestrianActionDesc',
       'Prepare

In [36]:
# Rename all columns
new_col_list = []
for item in crashes.columns:
	item = item.strip()
	item = item.replace('/', ' ')
	item = item.replace(' ', '_')
	item = item.replace('#', 'Num')
	item = item.replace('__', '_')
	item = item.capitalize()
	new_col_list.append(item)
	
# Reassign column names to new column names
crashes.columns = new_col_list
crashes.head()

,_id,Collision_id,Report_number,Report_version,Is_preliminary,Ncic_code,Crash_date_time,Crash_time_description,Beat,City_id,City_code,City_name,County_code,City_is_active,City_is_incorporated,Collision_type_code,Collision_type_description,Collision_type_other_desc,Day_of_week,Dispatchnotified,Hasphotographs,Hitrun,Isattachmentsmailed,Isdeleted,Ishighwayrelated,Istowaway,Judicialdistrict,Motorvehicleinvolvedwithcode,Motorvehicleinvolvedwithdesc,Motorvehicleinvolvedwithotherdesc,Numberinjured,Numberkilled,Weather_1,Weather_2,Road_condition_1,Road_condition_2,Special_condition,Lightingcode,Lightingdescription,Latitude,Longitude,Milepostdirection,Milepostdistance,Milepostmarker,Milepostunitofmeasure,Pedestrianactioncode,Pedestrianactiondesc,Prepareddate,Primary_collision_factor_code,Primary_collision_factor_violation,Primarycollisionfactoriscited,Primarycollisionpartynumber,Primaryroad,Reportingdistrict,Reportingdistrictcode,Revieweddate,Roadwaysurfacecode,Secondarydirection,Secondarydistance,Secondaryroad,Secondaryunitofmeasure,Sketchdesc,Trafficcontroldevicecode,Createddate,Modifieddate,Iscountyroad,Isfreeway,Chp555version,Isadditonalobjectstruck,Notificationdate,Notificationtimedescription,Hasdigitalmediafiles,Evidencenumber,Islocationrefertonarrative,Isaoionesameaslocation
0,1,4550264,250110005,1,False,4116,2025-01-10T08:28:00,828,A,"1,310.00","4,116.00",San Mateo,41.00,True,True,C,REAR END,NaN,Friday,NotApplicable,True,NaN,NaN,False,False,False,NORTHERN TRAFFIC,C,OTHER MOTOR VEHICLE,NaN,1.00,0.00,CLEAR,NaN,NO UNUSUAL CONDITIONS,NaN,,A,DAYLIGHT,37.57,-122.33,NaN,NaN,NaN,NaN,A,NO PEDESTRIANS INVOLVED,NaN,A,22350 VC,False,1.00,N EL CAMINO REAL,13A,NaN,NaN,A,NaN,NaN,TILTON AVENUE,NaN,NaN,A,2025-01-27T09:04:06,2025-01-27T09:04:06,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,4550263,9580-2025-00214,1,False,9580,2025-01-23T08:45:00,845,171,535.00,"1,942.00",Los Angeles,19.00,True,True,B,SIDE SWIPE,NaN,Thursday,NotApplicable,False,M,NaN,False,True,False,LOS ANGELES SUPERIOR COURT VAN NUYS COURTHOUSE...,C,OTHER MOTOR VEHICLE,NaN,0.00,0.00,CLEAR,NaN,NO UNUSUAL CONDITIONS,NaN,,A,DAYLIGHT,34.20,-118.40,NaN,NaN,NaN,NaN,A,NO PEDESTRIANS INVOLVED,2025-01-23T08:45:00,A,VC 22107,False,1.00,SR-170 S/B FROM SHERMAN WAY,NaN,NaN,2025-01-27T09:03:34,A,S,500.00,SHERMAN WAY,F,NaN,D,2025-01-27T09:03:36,2025-01-27T09:03:36,False,True,4,NaN,2025-01-23T08:47:00,0847,False,NaN,NaN,True
2,3,4550262,9765-2025-00066,1,False,9765,2025-01-15T12:30:00,1230,041,"1,701.00","5,606.00",Santa Paula,56.00,True,True,E,HIT OBJECT,NaN,Wednesday,NotApplicable,False,NaN,NaN,False,True,False,VENTURA SUPERIOR COURT VENTURA DIVISION,J,OTHER OBJECT,METAL OBJECT,0.00,0.00,WIND,NaN,NO UNUSUAL CONDITIONS,NaN,,A,DAYLIGHT,34.33,-119.09,NaN,NaN,NaN,NaN,A,NO PEDESTRIANS INVOLVED,2025-01-15T12:30:00,A,VC 22350,False,1.00,SR-126 E/B,NaN,NaN,2025-01-27T09:03:01,A,W,160.00,TODD LANE,F,NaN,D,2025-01-27T09:03:03,2025-01-27T09:03:03,False,True,4,NaN,2025-01-15T12:35:00,1235,True,EMBEDDED,NaN,True
3,4,4550261,9426-2025-00035,1,False,9426,2025-01-20T12:30:00,1230,460,397.00,"1,500.00",Unincorporated,15.00,True,False,D,BROADSIDE,NaN,Monday,No,False,NaN,NaN,False,False,True,KERN COUNTY SUPERIOR COURT - SHAFTER (MISDEMEA...,C,OTHER MOTOR VEHICLE,NaN,3.00,0.00,CLEAR,NaN,NO UNUSUAL CONDITIONS,NaN,,A,DAYLIGHT,35.62,-119.69,NaN,NaN,NaN,NaN,A,NO PEDESTRIANS INVOLVED,2025-01-20T12:30:00,A,VC 22450(a),False,1.00,SR-46 W/B,NaN,NaN,2025-01-27T09:02:37,A,NaN,NaN,MARTIN AVE,NaN,NaN,A,2025-01-27T09:02:47,2025-01-27T09:02:47,True,False,4,NaN,2025-01-20T12:32:00,1232,False,NaN,NaN,True
4,5,4550260,9580-2025-00216,1,False,9580,2025-01-23T10:30:00,1030,170,535.00,"1,942.00",Los Angeles,19.00,True,True,C,REAR END,NaN,Thursday,NotApplicable,False,NaN,NaN,False,True,False,VAN NUYS COURTHOUSE WEST,C,OTHER MOTOR VEHICLE,NaN,0.00,0.00,CLEAR,NaN,NO UNUSUAL CONDITIONS,NaN,,A,DAYLIGHT,34.20,-118.40,NaN,NaN,NaN,NaN,A,NO PEDESTRIANS INVOLVED,2025-01-23T10:30:00,A,VC 22350,False,1.00,SR-170 N/B TO SHERMAN WAY E/B,NaN,NaN,20

In [38]:
# View data types
crashes.dtypes

_id                                     int64
Collision_id                            int64
Report_number                          object
Report_version                          int64
Is_preliminary                           bool
Ncic_code                              object
Crash_date_time                        object
Crash_time_description                  int64
Beat                                   object
City_id                               float64
City_code                             float64
City_name                              object
County_code                           float64
City_is_active                         object
City_is_incorporated                   object
Collision_type_code                    object
Collision_type_description             object
Collision_type_other_desc              object
Day_of_week                            object
Dispatchnotified                       object
Hasphotographs                         object
Hitrun                            

In [ ]:
# Overview: length, non-null count, dtypes
crashes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195655 entries, 0 to 195654
Data columns (total 75 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   _id                                 195655 non-null  int64  
 1   Collision_id                        195655 non-null  int64  
 2   Report_number                       195617 non-null  object 
 3   Report_version                      195655 non-null  int64  
 4   Is_preliminary                      195655 non-null  bool   
 5   Ncic_code                           195655 non-null  object 
 6   Crash_date_time                     195655 non-null  object 
 7   Crash_time_description              195655 non-null  int64  
 8   Beat                                186144 non-null  object 
 9   City_id                             195627 non-null  float64
 10  City_code                           195627 non-null  float64
 11  City_name                 

In [ ]:
# Review columns with most missing values
crashes.isnull().sum().sort_values(ascending=False)

Isattachmentsmailed                   195655
Reportingdistrictcode                 195596
Milepostunitofmeasure                 195369
Sketchdesc                            195267
Milepostdirection                     195241
Milepostdistance                      195210
Milepostmarker                        195210
Road_condition_2                      195126
Collision_type_other_desc             192582
Weather_2                             190668
Islocationrefertonarrative            189355
Isadditonalobjectstruck               182970
Evidencenumber                        171846
Motorvehicleinvolvedwithotherdesc     161657
Hitrun                                149673
Reportingdistrict                     139063
Isaoionesameaslocation                 90439
Iscountyroad                           81581
Isfreeway                              81581
Notificationtimedescription            81344
Notificationdate                       81341
Hasdigitalmediafiles                   81308
Preparedda

In [43]:
# Drop 17 columns with excessive missing values (>70% null)
crashes_bx = crashes.drop(['Isattachmentsmailed', 'Reportingdistrictcode', 'Milepostunitofmeasure', 'Sketchdesc', 
                            'Milepostdirection', 'Milepostmarker', 'Hitrun', 'Reportingdistrict', 'Isaoionesameaslocation',
                            'Milepostdistance', 'Road_condition_2', 'Collision_type_other_desc', 'Weather_2', 'Islocationrefertonarrative', 
                            'Isadditonalobjectstruck', 'Evidencenumber', 'Motorvehicleinvolvedwithotherdesc'],
                            axis=1, errors='ignore')

# Further drop 8 irrelevant columns
crashes_bx.drop(['Isaoionesameaslocation', 'Iscountyroad', 'Hasdigitalmediafiles', 'Beat', 
                 'Secondarydirection', 'Secondarydistance', 'Secondaryunitofmeasure', 'Secondaryroad'],
                    axis=1, inplace=True, errors='ignore')

# Check resulting dataset shape
crashes_bx.shape

(195655, 51)

In [41]:
# Verify that collision_id is a good unique identifier
col_id_dupes = crashes_bx['Collision_id'].duplicated().sum()
print(col_id_dupes)

# Investigate duplicates (none)
col_dupe_rows = crashes_bx[crashes_bx['Collision_id'].duplicated(keep=False)]
print(col_dupe_rows.sort_values('Collision_id'))


0
Empty DataFrame
Columns: [_id, Collision_id, Report_number, Report_version, Is_preliminary, Ncic_code, Crash_date_time, Crash_time_description, City_id, City_code, City_name, County_code, City_is_active, City_is_incorporated, Collision_type_code, Collision_type_description, Day_of_week, Dispatchnotified, Hasphotographs, Isdeleted, Ishighwayrelated, Istowaway, Judicialdistrict, Motorvehicleinvolvedwithcode, Motorvehicleinvolvedwithdesc, Numberinjured, Numberkilled, Weather_1, Road_condition_1, Special_condition, Lightingcode, Lightingdescription, Latitude, Longitude, Pedestrianactioncode, Pedestrianactiondesc, Prepareddate, Primary_collision_factor_code, Primary_collision_factor_violation, Primarycollisionfactoriscited, Primarycollisionpartynumber, Primaryroad, Revieweddate, Roadwaysurfacecode, Trafficcontroldevicecode, Createddate, Modifieddate, Isfreeway, Chp555version, Notificationdate, Notificationtimedescription]
Index: []


In [42]:
# Investigate IsFreeWay vs IsHighwayRelated
print("Ishighwayrelated has: ", crashes_bx['Ishighwayrelated'].count(), "\nIsfreeway has: ", 
crashes_bx['Isfreeway'].count())

Ishighwayrelated has:  195654 
Isfreeway has:  114074


Based on above, disregard IsFreeWay column.

## Exploratory Data Analysis (EDA)

This section explores crash patterns in the 2025 YTD California Collision Data, focusing on both **frequency** (which events happen most often) and **severity** (which events result in the most injuries). 

The analysis is organized into two main sections:

- **Involved Party Analysis**: Identifies the most common collision partners (e.g., other motor vehicles, pedestrians, bicycles) and compares them by both number of crashes and average injuries 
- **Collision Type Analysis**: Examines the most frequent collision types (rear-end, sideswipe, broadside, etc.) and compares their injury severity

Both frequency and severity are visualized side by side to highlight important tradeoffs, such as:  
- Common but lower-risk events (e.g., parked vehicle crashes)
- Less common but higher-risk events (e.g., pedestrian, motorcycle, or broadside collisions)  

The goal of this EDA is not only to quantify patterns but also to **identify high-risk scenarios** that warrant further attention from traffic safety and vehicle design perspectives.

##### What is the most common involved party type in a collision, and how does this impact the average number of injuries per collision?

In [ ]:
# Collision count frequency of each involved party type 
crashes_bx['Motorvehicleinvolvedwithdesc'].value_counts()

Motorvehicleinvolvedwithdesc
OTHER MOTOR VEHICLE               123962
FIXED OBJECT                       32969
PARKED MOTOR VEHICLE               16106
OTHER OBJECT                        5404
PEDESTRIAN                          5191
BICYCLE                             4335
NON-COLLISION                       4305
MOTOR VEHICLE ON OTHER ROADWAY      1688
ANIMAL                              1128
TRAIN                                 63
Name: count, dtype: int64

In [44]:
# Bar chart: Frequency of involved party types in collisions

# Define color order to ensure consistent colors across plots
color_order = crashes_bx['Motorvehicleinvolvedwithdesc'].unique().tolist()

# Count involved party types
involved_type_counts = (
    crashes_bx['Motorvehicleinvolvedwithdesc']
    .value_counts(ascending=True)
    .reset_index()
)

involved_type_counts.columns = ['Involved Party', '# of Collisions']

# Create horizontal bar chart with Plotly
fig = px.bar(
    involved_type_counts,
    x='# of Collisions',
    y='Involved Party',
    orientation='h',
    title="Involved Party in Collisions - CA 2025 YTD Data From CCSR",
    labels={'# of Collisions': '# of Collisions', 'Involved Party': 'Involved Party'},
    color='Involved Party',
    category_orders={'Involved Party': color_order},
    width=800,
    height=600
)

fig.update_layout(yaxis={'categoryorder': 'array', 'categoryarray': involved_type_counts['Involved Party']}, showlegend=False)

fig.show()

This chart shows the distribution of 'Involved Party Type', which refers to the other party in the collision. 

The category of 'Other motor vehicles' is, by a wide margin, the most common category. The next three largest categories involve stationary objects: fixed object, parked motor vehicle, and other object. 

This highlights that while moving vehicles dominate collisions, stationary obstacles also play a significant role.  

Next, we explore how the average number of injuries varies by involved party type.

In [45]:
# Average number of injuries per involved party type
avg_injured_involved_party = (
    crashes_bx.groupby('Motorvehicleinvolvedwithdesc')['Numberinjured']
    .mean()
    .sort_values(ascending=False)
    .reset_index()
    .rename(columns={'Motorvehicleinvolvedwithdesc': 'Involved Party',
                     'Numberinjured': 'Avg Injured'})
)

# Bar chart of average injuries by involved party type
fig = px.bar(
    avg_injured_involved_party,
    x = 'Involved Party',
    y = 'Avg Injured',
    title = 'Average Number Injured by Involved Party',
    text = 'Avg Injured',
    color = 'Involved Party',
    category_orders={'Involved Party': color_order},
)

# Format text labels and axis
fig.update_traces(texttemplate='%{text:.2f}')
fig.update_layout(
    xaxis_title = 'Involved Party',
    yaxis_title = 'Average Number Injured',
    showlegend=False,
    xaxis={'categoryorder': 'array', 'categoryarray': avg_injured_involved_party['Involved Party']}

)

fig.show()

Pedestrians and bicycles account for the two highest average numbers of injuries per collision.  

In contrast, fixed objects, other objects, and parked motor vehicles — which were among the most common involved party types — fall into the bottom four for average injuries.  

In [ ]:
# Create summary table of average injuries by involved party type
avg_injured_involved_party_table = (
    crashes_bx
    .pivot_table(
        index='Motorvehicleinvolvedwithdesc',
        values='Numberinjured',
        aggfunc='mean'
    )
    .fillna(0)
    .rename(columns={'Numberinjured': 'Avg Injured'})
    .sort_values('Avg Injured', ascending=False)

)

avg_injured_involved_party_table.columns = ['Avg Injured']

avg_injured_involved_party_table

,Avg Injured
Motorvehicleinvolvedwithdesc,
PEDESTRIAN,0.97
BICYCLE,0.91
MOTOR VEHICLE ON OTHER ROADWAY,0.81
NON-COLLISION,0.75
TRAIN,0.75
OTHER MOTOR VEHICLE,0.64
FIXED OBJECT,0.39
OTHER OBJECT,0.27
PARKED MOTOR VEHICLE,0.19


In [47]:
# Combine collision counts with average injuries by involved party type

# Collision count per involved party 
num_involved_type = (
    crashes_bx['Motorvehicleinvolvedwithdesc']
    .value_counts()
    .reset_index(name='Num Collisions')
    .rename(columns={
        'Motorvehicleinvolvedwithdesc' : 'Involved Party'})
)

# Merge with avg injuries table 
involved_type = avg_injured_involved_party.merge(num_involved_type, on='Involved Party')

involved_type.head()

,Involved Party,Avg Injured,Num Collisions
0,PEDESTRIAN,0.97,5191
1,BICYCLE,0.91,4335
2,MOTOR VEHICLE ON OTHER ROADWAY,0.81,1688
3,NON-COLLISION,0.75,4305
4,TRAIN,0.75,63


In [49]:
# Top involved party types by number of collisions
involved_type.sort_values(ascending=False, by='Num Collisions').head()

,Involved Party,Avg Injured,Num Collisions
5,OTHER MOTOR VEHICLE,0.64,123962
6,FIXED OBJECT,0.39,32969
8,PARKED MOTOR VEHICLE,0.19,16106
7,OTHER OBJECT,0.27,5404
0,PEDESTRIAN,0.97,5191


In [50]:
# Scatter plot: collision frequency vs average injuries by involved party type
fig = px.scatter(
    involved_type,
    x='Num Collisions',
    y='Avg Injured',
    color='Involved Party',
    hover_name='Involved Party',
    title = 'Average Injured vs Number of Collisions by Involved Party',
    category_orders={'Involved Party': color_order},
    size='Num Collisions',
    size_max=100,
    width=1000,
    height=500
)

fig.update_layout(
    xaxis_title='Number of Collisions',
    yaxis_title='Average Number Injured',
    legend_title='Involved Party'
)

fig.show()

Previously, we examined involved party types separately using two bar charts: one for **number of collisions** and another for **average injuries**.  
This scatterplot brings both measures together:  
- **X-axis** = number of collisions  
- **Y-axis** = average number of injuries  
- **Dot size** = number of collisions (scaled)  

Key takeaways:  
- **Trains** rank in the top three for average injuries but occur very rarely.  
- **Pedestrians** are more concerning: they show the highest average injuries and also fall in the top five for collision frequency.  
- **Other motor vehicles** remain by far the most common involved party type, consistent with earlier charts.  

##### What is the most common collision type, and how does that impact average injuries per collision?  

In [53]:
# How many collisions were highway related vs not? 
crashes_bx['Ishighwayrelated'].value_counts()

Ishighwayrelated
False    111698
True      83956
Name: count, dtype: int64

How many collisions were highway related vs not? 

Not Highway Related: 98,817

Highway Related: 76, 123

In [25]:
# What was the most common collision type?
crashes_bx['Collision_type_description'].value_counts()

Collision_type_description
REAR END              59476
SIDE SWIPE            44290
HIT OBJECT            35640
BROADSIDE             32670
HEAD-ON                8217
OTHER                  5559
VEHICLE/PEDESTRAIN     4954
OVERTURNED             4420
Name: count, dtype: int64

In [54]:
# Bar chart: count by collision types

# Define color order to ensure consistent colors across plots
col_order = crashes_bx['Collision_type_description'].unique().tolist()

# Count by collision types 
collision_type_counts = (
    crashes_bx['Collision_type_description']
    .value_counts(ascending=True)
    .reset_index()
)

collision_type_counts.columns = ['Collision Type', '# of Collisions']

# Horizontal bar chart of collision types
fig = px.bar(
    collision_type_counts,
    x='# of Collisions',
    y='Collision Type',
    orientation='h',
    title="Collision Type - CA 2025 YTD Data From CCSR",
    labels={'# of Collisions': '# of Collisions', 'Collision Type': 'Collision Type'},
    color='Collision Type',
    category_orders={'Collision Type': col_order},
    width=800,
    height=600
)

fig.update_layout(yaxis={'categoryorder': 'array', 'categoryarray': collision_type_counts['Collision Type']}, showlegend=False)

fig.show()

**Most common collision types:** rear end (highest), side swipe, hit object, and broadside.

**What is a “broadside”?** A crash where the front of one vehicle strikes the side of another, forming a “T” (also called a T-bone or side-impact collision). Broadsides often occur at intersections when:  
1) a driver runs a red light or stop sign,  
2) a vehicle turns across oncoming traffic, or  
3) a driver fails to yield

Because the sides of vehicles typically offer less structural protection than the front or rear, broadsides can be more dangerous. In the next section, we examine **average injuries by collision type** to see whether broadsides exhibit a relatively higher injury rate.

In [55]:
# Average injuries by collision type
avg_injured_collision_type = (
    crashes_bx.groupby('Collision_type_description')['Numberinjured']
    .mean()
    .sort_values(ascending=False)
    .reset_index()
    .rename(columns={'Collision_type_description': 'Collision Type',
                     'Numberinjured': 'Avg Injured'})
)

# Bar chart: average injuries per collision type
fig = px.bar(
    avg_injured_collision_type,
    x = 'Collision Type',
    y = 'Avg Injured',
    title = 'Average Number Injured by Collision Type',
    text = 'Avg Injured',
    color = 'Collision Type',
    category_orders={'Collision Type': col_order},
)

fig.update_traces(texttemplate='%{text:.2f}')
fig.update_layout(
    xaxis_title = 'Collision Type',
    yaxis_title = 'Average Number Injured',
    showlegend=False,
    xaxis={'categoryorder': 'array', 'categoryarray': avg_injured_collision_type['Collision Type']}

)

fig.show()

The data show that **vehicle–pedestrian collisions** have the highest average number of injuries per crash, making them the most dangerous type overall.  

As expected, **broadsides (T-bone collisions)** also rank near the top, confirming their higher injury risk compared to most other types.  

Additionally, **head on** and **overturned collisions** appear among the most severe.

In [56]:
# Combine frequency and severity for collision types

# Count per collision type
num_collision_type = (
    crashes_bx['Collision_type_description']
    .value_counts()
    .reset_index(name='Num Collisions')
    .rename(columns={
        'Collision_type_description' : 'Collision Type'})
)

# Merge with average injuries per type
collision_type = avg_injured_collision_type.merge(num_collision_type, on='Collision Type')

# Show top 5 collision types by frequency
collision_type.sort_values(ascending=False, by='Num Collisions').head()

,Collision Type,Avg Injured,Num Collisions
4,REAR END,0.60,59476
7,SIDE SWIPE,0.28,44290
6,HIT OBJECT,0.36,35640
1,BROADSIDE,0.94,32670
2,HEAD-ON,0.93,8217


In [57]:
# Scatter plot: Collision type frequency vs average injuries
fig = px.scatter(
    collision_type,
    x='Num Collisions',
    y='Avg Injured',
    color='Collision Type',
    hover_name='Collision Type',
    title = 'Average Injured vs Number of Collisions by Collision Type',
    category_orders={'Collision Type': col_order},
    size='Num Collisions',
    size_max=40
)

fig.update_layout(
    xaxis_title='Number of Collisions',
    yaxis_title='Average Number Injured',
    legend_title='Collision Type'
)

fig.show()

Previously, we examined collision types separately using two bar charts: one for **number of collisions** and another for **average injuries**.  
This scatterplot combines both dimensions:  
- **X-axis** = number of collisions  
- **Y-axis** = average number of injuries  
- **Dot size** = number of collisions (scaled)  

Key takeaway: **Broadsides (T-bone collisions)** stand out with both a high average number of injuries and a high frequency (top four by count). Given this impact, we researched strategies to reduce the risks associated with broadside crashes.  

**Prevention strategies**  
- Practice defensive driving at intersections
- Slow down and scan both directions, even with a green light  
- Avoid accelerating immediately when the light turns green, reducing exposure to red-light runners
- Follow traffic rules: avoid running red lights or rolling through stop signs

**Impact reduction strategies**  
- Vehicle design: side-curtain airbags, reinforced door beams, and strong side-impact safety ratings 
- Driver/rider behavior: maintain upright seating positions and reduce speed around busy intersections  
- Further research opportunity: analyze whether certain makes/models with better side-impact protection correlate with lower injury rates in broadside collisions

## Key Insights

From the 2025 YTD California collision data, several patterns emerge:

- **Passenger vehicles** dominate collision counts, but **pedestrian and bicycle crashes** stand out as the most dangerous, with the highest average number of injuries per collision.  
- **Broadsides (T-bone collisions)** combine both high frequency and high injury severity, making them a critical target for prevention strategies.  
- **Head-on and overturned collisions** are less frequent but remain among the most severe crash types.  
- **Stationary object collisions** (fixed objects, parked vehicles) are common, but generally result in fewer injuries.  

Together, these findings highlight the importance of focusing not just on *common* crash types, but also on those that are disproportionately severe. This has implications for **traffic safety policy, intersection design, and vehicle safety features** (e.g., side-impact protection).